In [1]:
import numpy as np
import pandas as pd
import torch
from fourier_conv import ConvFourier
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from utils import get_generator
from matplotlib import pyplot as plt



In [2]:
class Model(nn.Module):
    
    def __init__(self, fourierPrecision=6):
        super(Model, self).__init__()
        self.Fl1 = nn.Sequential(
                        ConvFourier(in_channels=1, out_channels=3, kernel_size=[3,3],sequence_length=fourierPrecision, stride=1,padding=0),
                        nn.Sigmoid()
                )
        self.Fl2 = nn.Sequential(
                        ConvFourier(in_channels=3, out_channels=6, kernel_size=[3,3],sequence_length=fourierPrecision, stride=1,padding=0),
                        nn.Sigmoid()
                )
        self.Fl3 = nn.Sequential(
                        ConvFourier(in_channels=6, out_channels=10, kernel_size=[4,4],sequence_length=fourierPrecision, stride=1, padding=0),
                        nn.Sigmoid()
                )
        self.fcl1 = nn.Softmax()
    
    def forward(self, img):
        img=self.Fl1(img)
        img=self.Fl2(img)
        img=self.Fl3(img)
        img = img.reshape(img.size(0), -1)
        return self.fcl1(img)

In [3]:
## Hyper-Parameters
batch_size=2
workers=1
dataset = get_generator(batch_size=batch_size, num_workers=workers)
lr = 1e-3
wd = 0.0000001

model=Model()
criterion = nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=wd, amsgrad=True)

In [4]:
for i, data in enumerate(dataset):
    img, labels=data
    img=torch.unsqueeze(img.float(), 1)
    preds=model(img)
    loss=criterion(preds, labels)
    if(i%10==0):
        print(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

/home/abjb/workspace/fourierConvNet/Research-Fourier-Convolutions/fourier_conv.py:102: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out_tensor = torch.tensor(torch.zeros((batch_size,out_channels,out_height, out_width), dtype=torch.float32))
/home/abjb/workspace/fourierConvNet/Research-Fourier-Convolutions/fourier_conv.py:118: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out_tensor = torch.tensor(torch.zeros(input.shape), dtype= torch.float32)
/home/abjb/.virtualenvs/ML/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(2.3032, grad_fn=<NllLossBackward>)
tensor(2.2673, grad_fn=<NllLossBackward>)
tensor(2.3345, grad_fn=<NllLossBackward>)
tensor(2.3341, grad_fn=<NllLossBackward>)
tensor(2.3314, grad_fn=<NllLossBackward>)


KeyboardInterrupt: 